In [ ]:
import torch as th

In [ ]:
derivative = th.randn(10, 600, 10)
dim = 1
dx = 10
sizes = derivative.size()
n = derivative.size()[dim]

In [ ]:
    evens = th.arange(0, n, 2)
    odds = th.arange(1, n, 2)

    even_derivative = th.index_select(derivative, dim, evens)
    odd_derivative = th.index_select(derivative, dim, odds)

    shift_odd_derivative = th.cat(
        (
            th.zeros(*[1 if i == dim else s for i, s in enumerate(sizes)]),
            odd_derivative,
        ),
        dim=dim,
    )

    even_primitive = 0. + dx / 3 * (
            (
                    2 * even_derivative
                    + 4 * th.index_select(shift_odd_derivative, dim=dim,
                                          index=th.arange(0, even_derivative.size()[dim]))
            ).cumsum(dim)
            - th.select(even_derivative, dim, 0).unsqueeze(dim)
            - th.select(even_derivative, dim, 0).unsqueeze(dim)
    )

    odd_primitive = (dx / 3) * (
            (
                    2 * odd_derivative
                    + 4 * th.index_select(even_derivative, dim=dim,
                                          index=th.arange(0, odd_derivative.size()[dim]))
            ).cumsum(dim)
            - 4 * th.select(even_derivative, dim, 0).unsqueeze(dim)
            - th.select(odd_derivative, dim, 0).unsqueeze(dim)
            - odd_derivative
    )

In [ ]:
odd_primitive

In [ ]:
sizes = derivative.size()
n = derivative.size()[dim]

evens = th.arange(0, n, 2)
odds = th.arange(1, n, 2)

even_derivative = th.index_select(derivative, dim, evens)
odd_derivative = th.index_select(derivative, dim, odds)

In [ ]:
even_derivative.size(), odd_derivative.size(), odds.size(), evens.size()

In [ ]:
even_primitive = 0 + dx / 3 * (
        (
            2 * even_derivative
            + 4 * th.index_select(odd_derivative, dim=dim, index=th.arange(0, even_derivative.size()[dim]))
        ).cumsum(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
    )

In [ ]:
odd_primitive = (dx / 3) * (
        (
            2 * odd_derivative
            + 4 * th.index_select(even_derivative, dim=dim, index=th.arange(0, odd_derivative.size()[dim]))
        ).cumsum(dim)
        - 4 * th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(odd_derivative, dim, 0).unsqueeze(dim)
        - odd_derivative
    )

In [ ]:
    sizes = derivative.size()
    n = derivative.size()[dim]

    evens = th.arange(0, n, 2)
    odds = th.arange(1, n, 2)

    even_derivative = th.index_select(derivative, dim, evens)
    odd_derivative = th.index_select(derivative, dim, odds)

    even_primitive = first_primitive + dx / 3 * (
        (
            2 * even_derivative
            + 4 * th.index_select(odd_derivative, dim=dim, index=th.arange(0, even_derivative.size()[dim]))
        ).cumsum(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
    )

    odd_primitive = (dx / 3) * (
        (
            2 * odd_derivative
            + 4 * th.index_select(even_derivative, dim=dim, index=th.arange(0, odd_derivative.size()[dim]))
        ).cumsum(dim)
        - 4 * th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(odd_derivative, dim, 0).unsqueeze(dim)
        - odd_derivative
    )

    odd_primitive += first_primitive + dx / 12 * (
        5 * derivative[0] + 8 * derivative[1] - derivative[2]
    )

    primitive = th.zeros_like(derivative)
    primitive[evens] = even_primitive
    primitive[odds] = odd_primitive

In [ ]:
def simpson(
    first_primitive: float, derivative: th.Tensor, dim: int, dx: float,
) -> th.Tensor:

    sizes = derivative.size()
    n = derivative.size()[dim]

    evens = th.arange(0, n, 2)
    odds = th.arange(1, n, 2)

    even_derivative = th.index_select(derivative, dim, evens)
    odd_derivative = th.index_select(derivative, dim, odds)
    
    shift_odd_derivative = th.cat(
        (
            th.zeros(*[1 if i == dim else s for i, s in enumerate(sizes)]),
            odd_derivative,
        ),
        dim=dim,
    )

    even_primitive = first_primitive + dx / 3 * (
        (
            2 * even_derivative
            + 4 * th.index_select(shift_odd_derivative, dim=dim, index=th.arange(0, even_derivative.size()[dim]))
        ).cumsum(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(even_derivative, dim, 0).unsqueeze(dim)
    )

    odd_primitive = (dx / 3) * (
        (
            2 * odd_derivative
            + 4 * th.index_select(even_derivative, dim=dim, index=th.arange(0, odd_derivative.size()[dim]))
        ).cumsum(dim)
        - 4 * th.select(even_derivative, dim, 0).unsqueeze(dim)
        - th.select(odd_derivative, dim, 0).unsqueeze(dim)
        - odd_derivative
    )

    odd_primitive += first_primitive + dx / 12 * (
        5 * th.select(derivative, dim, 0) + 8 * th.select(derivative, dim, 1) - th.select(derivative, dim, 2)
    ).unsqueeze(dim)

    primitive = th.zeros_like(derivative)
    
    evens = evens.view(*[-1 if i == dim else 1 for i in range(len(sizes))])
    odds = odds.view(*[-1 if i == dim else 1 for i in range(len(sizes))])
    
    primitive.scatter_(dim, evens, even_primitive)
    primitive.scatter_(dim, odds, odd_primitive)

    return primitive

In [ ]:
simpson(0, th.randn(300, 100, 200), 1, 10)